In [27]:
import sys
sys.path.append('/scratch/ty296/CT_MPS_mini')
from read_hdf5_func import von_neumann_entropy_sv
import h5py
import numpy as np

In [28]:
# Read the HDF5 file
filename = 'hdf5_data_combined/sv_combined_p_ctrl0.0.h5'


In [33]:
p_proj_list = np.linspace(0.0,1.0,20)
p_target = p_proj_list[3]
print(p_target)

0.15789473684210525


In [34]:
counter = 0
with h5py.File(filename, 'r') as f:
    print("Keys in HDF5 file:")
    keys_list = list(f.keys())[:]
    for key in keys_list:
        L = f[key].attrs['L']
        p_proj = f[key].attrs['p_proj']
        if L == 24 and p_proj == p_target:
            counter += 1
            # print(f[key].attrs['L'])
            print(f"  {L}: {len(f[key][()])}: {f[key][()]}")

    print(f"Found {counter} datasets")
    # # If you want to see the structure of each dataset
    # print("\nDataset shapes and types:")
    # for key in f.keys():
    #     dataset = f[key]
    #     print(f"  {key}: shape={dataset.shape}, dtype={dataset.dtype}")


Keys in HDF5 file:
  24: 50: [0.54208519 0.36536749 0.31949402 0.28142587 0.25485741 0.21023457
 0.19918123 0.18921193 0.17051983 0.14737817 0.13374535 0.13150481
 0.12238167 0.11219519 0.10221553 0.09446424 0.0901911  0.08465564
 0.08008886 0.07760758 0.07122087 0.06648207 0.06435009 0.06102806
 0.05886352 0.05469046 0.05286233 0.04914873 0.0454011  0.04334405
 0.03957284 0.03728281 0.03486822 0.03375536 0.03221679 0.02993586
 0.02841134 0.02704118 0.02603089 0.02438028 0.02229645 0.02107151
 0.01950456 0.01889267 0.0181321  0.01698954 0.01611278 0.0144393
 0.01331506 0.0115784 ]
  24: 50: [0.44785843 0.37211694 0.3511086  0.27858504 0.25875757 0.22076392
 0.20592893 0.18237211 0.17131043 0.16854629 0.1468589  0.14038039
 0.12978382 0.12379005 0.11588044 0.10158856 0.09688562 0.08989723
 0.0801463  0.07417623 0.06782644 0.0639908  0.05623998 0.05289138
 0.04892613 0.04688293 0.04434374 0.04291119 0.03946747 0.03544555
 0.03350608 0.03098357 0.02963486 0.02812738 0.02639049 0.02446524
